In [102]:
import requests
import warnings
#from tqdm import tqdm, trange
import pandas as pd 
import streamlit as st 
import yfinance as yf
from datetime import datetime
from dateutil.relativedelta import relativedelta
import plotly_express  as px
import plotly.graph_objects as go
import random
import unittest
import tables
import pickle
import pyarrow.parquet as pq
import pyarrow as pa
import plotly.io as pio
import math

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [119]:
pio.templates.default = "plotly_dark"

In [185]:
ticker = yf.Ticker("AAPL")
cash_flow_stmnt = ((ticker.cash_flow).T)

balance_sheet_stmnt = (ticker.balance_sheet).T
info = ticker.info

In [184]:
industry = info.get('industry')
sector = info.get('sector')
description = info.get('longBusinessSummary')
beta = info.get('beta')
sharesOutstanding =  info.get('sharesOutstanding')
name = info.get('longName')
currentPrice = info.get('currentPrice')
targetHighPrice = info.get('targetHighPrice')
targetLowPrice = info.get('targetLowPrice')
targetMeanPrice = info.get('targetMeanPrice')


In [186]:
info

{'address1': 'One Apple Park Way',
 'city': 'Cupertino',
 'state': 'CA',
 'zip': '95014',
 'country': 'United States',
 'phone': '(408) 996-1010',
 'website': 'https://www.apple.com',
 'industry': 'Consumer Electronics',
 'industryKey': 'consumer-electronics',
 'industryDisp': 'Consumer Electronics',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and p

In [187]:
pb = info.get('priceToBook')
fPE = info.get('forwardPE')
tPE = info.get('trailingPE')
evebitda = info.get('enterpriseToEbitda')
evrevenue = info.get('enterpriseToRevenue')
rps = info.get('currentPrice')/info.get('revenuePerShare')

In [178]:
target_ticker_string = 'AAPL'

In [189]:
income_stmnt.columns.to_list()

['Tax Effect Of Unusual Items',
 'Tax Rate For Calcs',
 'Normalized EBITDA',
 'Net Income From Continuing Operation Net Minority Interest',
 'Reconciled Depreciation',
 'Reconciled Cost Of Revenue',
 'EBITDA',
 'EBIT',
 'Net Interest Income',
 'Interest Expense',
 'Interest Income',
 'Normalized Income',
 'Net Income From Continuing And Discontinued Operation',
 'Total Expenses',
 'Total Operating Income As Reported',
 'Diluted Average Shares',
 'Basic Average Shares',
 'Diluted EPS',
 'Basic EPS',
 'Diluted NI Availto Com Stockholders',
 'Net Income Common Stockholders',
 'Net Income',
 'Net Income Including Noncontrolling Interests',
 'Net Income Continuous Operations',
 'Tax Provision',
 'Pretax Income',
 'Other Income Expense',
 'Other Non Operating Income Expenses',
 'Net Non Operating Interest Income Expense',
 'Interest Expense Non Operating',
 'Interest Income Non Operating',
 'Operating Income',
 'Operating Expense',
 'Research And Development',
 'Selling General And Administr

In [182]:
target_ticker = yf.Ticker(target_ticker_string)
income_stmnt = (target_ticker.income_stmt).T
target_price_df = pd.DataFrame(target_ticker.history(start ='2020-01-01')['Close'])
target_price_df.index = target_price_df.index.date 
is_dfL = (income_stmnt)

is_dfL['Diluted Average Shares'] = pd.to_numeric(is_dfL['Diluted Average Shares'], errors='coerce')
st.markdown((is_dfL['Diluted Average Shares'].isna().sum()))
is_dfL['Diluted Average Shares'] = is_dfL['Diluted Average Shares'].fillna(method='bfill',)
st.markdown((is_dfL['Diluted Average Shares'].isna().sum()))

is_dfL['RPS'] = is_dfL['Total Revenue']/is_dfL['Diluted Average Shares']
is_dfL['EPS'] = is_dfL['Net Income']/is_dfL['Diluted Average Shares']
# Step 2: Reindex EPS to match daily prices and forward-fill values
eps_daily = is_dfL["EPS"].reindex(target_price_df.index, method="ffill")
rps_daily = is_dfL["RPS"].reindex(target_price_df.index, method="ffill")
# Step 3: Merge with Daily Prices
target_price_df["EPS"] = eps_daily
target_price_df["RPS"] = rps_daily
target_price_df["PE_Ratio"] = target_price_df["Close"] / target_price_df["EPS"]
target_price_df["RPS_Ratio"] = target_price_df["Close"] / target_price_df["RPS"]


pe_multiples = pd.DataFrame()
pe_multiples.index = target_price_df.index
pe_multiples[f'{target_ticker_string}'] = target_price_df["PE_Ratio"]

pr_multiples = pd.DataFrame()
pr_multiples.index = target_price_df.index
pr_multiples[f'{target_ticker_string}'] = target_price_df["RPS_Ratio"]
target_price_df

/var/folders/hs/v5c0lrnj61z3tp745vdhh0m40000gn/T/ipykernel_91556/200583938.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  is_dfL['Diluted Average Shares'] = is_dfL['Diluted Average Shares'].fillna(method='bfill',)


,Close,EPS,RPS,PE_Ratio,RPS_Ratio
2020-01-02,72.796013,NaN,NaN,NaN,NaN
2020-01-03,72.088303,NaN,NaN,NaN,NaN
2020-01-06,72.662704,NaN,NaN,NaN,NaN
2020-01-07,72.320961,NaN,NaN,NaN,NaN
2020-01-08,73.484352,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2025-01-27,229.860001,NaN,NaN,NaN,NaN
2025-01-28,238.259995,NaN,NaN,NaN,NaN
2025-01-29,239.360001,NaN,NaN,NaN,NaN
2025-01-30,237.589996,NaN,NaN,NaN,NaN


In [183]:
import yfinance as yf
import pandas as pd

# Define the target ticker
target_ticker_string = 'AAPL'
target_ticker = yf.Ticker(target_ticker_string)

# Step 1: Fetch historical stock prices
target_price_df = pd.DataFrame(target_ticker.history(start='2020-01-01')['Close'])
target_price_df.index = target_price_df.index.date  # Format index as date only

# Step 2: Fetch historical income statement & shares outstanding
income_stmnt = target_ticker.income_stmt.T  # Transpose for easier access
is_dfL = income_stmnt

# Step 3: Forward-fill missing values in the diluted shares column
is_dfL['Diluted Average Shares'].fillna(method='bfill', inplace=True)

# Step 4: Calculate EPS (Net Income / Diluted Shares)
is_dfL['EPS'] = is_dfL['Net Income'] / is_dfL['Diluted Average Shares']

# Step 5: Compute Trailing 12-Month (TTM) EPS
# Get the sum of the last 4 quarters of EPS
is_dfL['TTM_EPS'] = is_dfL['EPS'].rolling(window=4, min_periods=1).sum()

# Step 6: Merge historical stock prices with TTM EPS
pe_df = target_price_df.merge(is_dfL[['TTM_EPS']], left_index=True, right_index=True, how='left')

# Step 7: Calculate P/E Ratio (Price / TTM EPS)
pe_df['PE_Ratio'] = pe_df['Close'] / pe_df['TTM_EPS']


pe_df

/var/folders/hs/v5c0lrnj61z3tp745vdhh0m40000gn/T/ipykernel_91556/2601786498.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  is_dfL['Diluted Average Shares'].fillna(method='bfill', inplace=True)
/var/folders/hs/v5c0lrnj61z3tp745vdhh0m40000gn/T/ipykernel_91556/2601786498.py:17: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  is_dfL['Diluted Average Shares'].fillna(method='bfill', inplace=True)
/var/folders/hs/v5c0lrnj61z3tp745vdhh0m40000g

,Close,TTM_EPS,PE_Ratio
2020-01-02,72.796013,NaN,NaN
2020-01-03,72.088295,NaN,NaN
2020-01-06,72.662712,NaN,NaN
2020-01-07,72.320976,NaN,NaN
2020-01-08,73.484337,NaN,NaN
...,...,...,...
2025-01-27,229.860001,NaN,NaN
2025-01-28,238.259995,NaN,NaN
2025-01-29,239.360001,NaN,NaN
2025-01-30,237.589996,NaN,NaN
